# Builder (Polyglot)

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.2/lib/netstandard2.0/Argu.dll"

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common
open SpiralFileSystem.Operators

## buildProject

In [ ]:
let inline buildProject runtime outputDir path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let extension = fullPath |> System.IO.Path.GetExtension

    trace Debug
        (fun () -> "buildProject")
        (fun () -> $"fullPath: {fullPath} / {_locals ()}")

    match extension with
    | ".fsproj" -> ()
    | _ -> failwith "Invalid project file"

    let runtimes =
        runtime
        |> Option.map List.singleton
        |> Option.defaultValue [ "linux-x64"; "win-x64" ]

    let outputDir = outputDir |> Option.defaultValue "dist"

    return!
        runtimes
        |> List.map (fun runtime -> async {
            let command = $@"dotnet publish ""{path}"" --configuration Release --output ""{outputDir}"" --runtime {runtime}"
            let! exitCode, _result =
                SpiralRuntime.execution_options (fun x ->
                    { x with
                        l1 = command
                        l6 = Some fileDir
                    }
                )
                |> SpiralRuntime.execute_with_options_async
            return exitCode
        })
        |> Async.Sequential
        |> Async.map Array.sum
}

## persistCodeProject

In [ ]:
let inline persistCodeProject packages modules name hash code = async {
    trace Debug
        (fun () -> "persistCodeProject")
        (fun () -> $"packages: {packages} / modules: {modules} / name: {name} / hash: {hash} / code.Length: {code |> String.length} / {_locals ()}")

    let workspaceRoot = SpiralFileSystem.get_workspace_root ()

    let targetDir =
        let targetDir = workspaceRoot </> "target/polyglot/builder" </> name
        match hash with
        | Some hash -> targetDir </> "packages" </> hash
        | None -> targetDir
    System.IO.Directory.CreateDirectory targetDir |> ignore

    let filePath = targetDir </> $"{name}.fs" |> System.IO.Path.GetFullPath
    do! code |> SpiralFileSystem.write_all_text_exists filePath

    let modulesCode =
        modules
        |> List.map (fun path -> $"""<Compile Include="{workspaceRoot </> path}" />""")
        |> SpiralSm.concat "\n        "

    let fsprojPath = targetDir </> $"{name}.fsproj"
    let fsprojCode = $"""<Project Sdk="Microsoft.NET.Sdk">
    <PropertyGroup>
        <TargetFramework>net9.0</TargetFramework>
        <LangVersion>preview</LangVersion>
        <RollForward>Major</RollForward>
        <TargetLatestRuntimePatch>true</TargetLatestRuntimePatch>
        <PublishAot>false</PublishAot>
        <PublishTrimmed>false</PublishTrimmed>
        <PublishSingleFile>true</PublishSingleFile>
        <SelfContained>true</SelfContained>
        <Version>0.0.1-alpha.1</Version>
        <OutputType>Exe</OutputType>
    </PropertyGroup>

    <ItemGroup>
        {modulesCode}
        <Compile Include="{filePath}" />
    </ItemGroup>

    <Import Project="{workspaceRoot}/.paket/Paket.Restore.targets" />
</Project>
"""
    do! fsprojCode |> SpiralFileSystem.write_all_text_exists fsprojPath

    let paketReferencesPath = targetDir </> "paket.references"
    let paketReferencesCode =
        "FSharp.Core" :: packages
        |> SpiralSm.concat "\n"
    do! paketReferencesCode |> SpiralFileSystem.write_all_text_exists paketReferencesPath

    return fsprojPath
}

## buildCode

In [ ]:
let inline buildCode runtime packages modules outputDir name code = async {
    let! fsprojPath = code |> persistCodeProject packages modules name None
    let! exitCode = fsprojPath |> buildProject runtime outputDir
    if exitCode > 0 then
        let! fsprojText = fsprojPath |> SpiralFileSystem.read_all_text_async
        trace Critical
            (fun () -> "buildCode")
            (fun () -> $"code: {code |> SpiralSm.ellipsis_end 400} / fsprojText: {fsprojText} / {_locals ()}")
    return exitCode
}

In [ ]:
//// test

"1 + 1 |> ignore"
|> buildCode None [] [] None "test1"
|> Async.runWithTimeout 180000
|> _assertEqual (Some 0)

00:00:02   debug #1 persistCodeProject / packages: [] / modules: [] / name: test1 / hash:  / code.Length: 15
00:00:02   debug #2 buildProject / fullPath: C:\home\git\polyglot\target\polyglot\builder\test1\test1.fsproj
00:00:04   debug #1 execute_with_options_async / options: struct (None,
        "dotnet publish "C:\home\git\polyglot\target/polyglot/builder\test1\test1.fsproj" --configuration Release --output "dist" --runtime linux-x64",
        [||], None, None, true,
        Some "C:\home\git\polyglot\target\polyglot\builder\test1")
00:00:04 verbose #2 > MSBuild version 17.10.0-preview-24101-01+07fd5d51f for .NET
00:00:05 verbose #3 >   Determining projects to restore...
00:00:07 verbose #4 >   Restored C:\home\git\polyglot\target\polyglot\builder\test1\test1.fsproj (in 1.23 sec).
00:00:07 verbose #5 > C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\9.0.100-preview.1.24101.2\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(313,5): message NETSD

In [ ]:
//// test

"1 + a |> ignore"
|> buildCode None [] [] None "test2"
|> Async.runWithTimeout 180000
|> _assertEqual (Some 2)

00:00:17   debug #3 persistCodeProject / packages: [] / modules: [] / name: test2 / hash:  / code.Length: 15
00:00:17   debug #4 buildProject / fullPath: C:\home\git\polyglot\target\polyglot\builder\test2\test2.fsproj
00:00:19   debug #19 execute_with_options_async / options: struct (None,
        "dotnet publish "C:\home\git\polyglot\target/polyglot/builder\test2\test2.fsproj" --configuration Release --output "dist" --runtime linux-x64",
        [||], None, None, true,
        Some "C:\home\git\polyglot\target\polyglot\builder\test2")
00:00:19 verbose #20 > MSBuild version 17.10.0-preview-24101-01+07fd5d51f for .NET
00:00:19 verbose #21 >   Determining projects to restore...
00:00:20 verbose #22 >   Restored C:\home\git\polyglot\target\polyglot\builder\test2\test2.fsproj (in 331 ms).
00:00:20 verbose #23 > C:\Users\i574n\scoop\apps\dotnet-sdk-preview\current\sdk\9.0.100-preview.1.24101.2\Sdks\Microsoft.NET.Sdk\targets\Microsoft.NET.RuntimeIdentifierInference.targets(313,5): message NE

## readFile

In [ ]:
let inline readFile path = async {
    let! code = path |> SpiralFileSystem.read_all_text_async

    let code = System.Text.RegularExpressions.Regex.Replace (
        code,
        @"( *)(let\s+main\s+.*?\s*=)",
        fun m -> m.Groups.[1].Value + "[<EntryPoint>]\n" + m.Groups.[1].Value + m.Groups.[2].Value
    )

    let codeTrim = code |> SpiralSm.trim_end [||]
    return
        if codeTrim |> SpiralSm.ends_with "\n()"
        then codeTrim |> SpiralSm.slice 0 ((codeTrim |> String.length) - 3)
        else code
}

## buildFile

In [ ]:
let inline buildFile runtime packages modules path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let dir = fullPath |> System.IO.Path.GetDirectoryName
    let name = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> readFile
    return! code |> buildCode runtime packages modules (dir </> "dist" |> Some) name
}

## persistFile

In [ ]:
let inline persistFile packages modules path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let name = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> readFile
    return! code |> persistCodeProject packages modules name None
}

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce>] Path of path : string
    | [<Argu.ArguAttributes.Unique>] Packages of packages : string list
    | [<Argu.ArguAttributes.Unique>] Modules of modules : string list
    | [<Argu.ArguAttributes.Unique>] Runtime of runtime : string
    | [<Argu.ArguAttributes.Unique>] Persist_Only

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Path _ -> nameof Path
            | Packages _ -> nameof Packages
            | Modules _ -> nameof Modules
            | Runtime _ -> nameof Runtime
            | Persist_Only -> nameof Persist_Only

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

"USAGE: dotnet-repl [--help] [--packages [<packages>...]]
                   [--modules [<modules>...]] [--runtime <runtime>]
                   [--persist-only] <path>

PATH:

    <path>                Path

OPTIONS:

    --packages [<packages>...]
                          Packages
    --modules [<modules>...]
                          Modules
    --runtime <runtime>   Runtime
    --persist-only        Persist_Only
    --help                display this list of options.
"


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let path =
        match argsMap.[nameof Arguments.Path] with
        | [ Arguments.Path path ] -> Some path
        | _ -> None
        |> Option.get

    let packages =
        match argsMap |> Map.tryFind (nameof Arguments.Packages) with
        | Some [ Arguments.Packages packages ] -> packages
        | _ -> []

    let modules =
        match argsMap |> Map.tryFind (nameof Arguments.Modules) with
        | Some [ Arguments.Modules modules ] -> modules
        | _ -> []

    let runtime =
        match argsMap |> Map.tryFind (nameof Arguments.Runtime) with
        | Some [ Arguments.Runtime runtime ] -> Some runtime
        | _ -> None

    let persistOnly = argsMap |> Map.containsKey (nameof Arguments.Persist_Only)

    if persistOnly
    then path |> persistFile packages modules |> Async.map (fun _ -> 0)
    else path |> buildFile runtime packages modules
    |> Async.runWithTimeout (60000 * 60)
    |> function
        | Some exitCode -> exitCode
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> SpiralRuntime.split_args
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0

00:00:24   debug #6 persistCodeProject / packages: [Argu; FSharp.Control.AsyncSeq; System.Reactive.Linq] / modules: [lib/spiral/common.fsx; lib/spiral/sm.fsx; lib/spiral/crypto.fsx; ... ] / name: Builder / hash:  / code.Length: 7651
00:00:24   debug #7 buildProject / fullPath: C:\home\git\polyglot\target\polyglot\builder\Builder\Builder.fsproj
00:00:26   debug #33 execute_with_options_async / options: struct (None,
        "dotnet publish "C:\home\git\polyglot\target/polyglot/builder\Builder\Builder.fsproj" --configuration Release --output "C:\home\git\polyglot\apps\builder\dist" --runtime linux-x64",
        [||], None, None, true,
        Some "C:\home\git\polyglot\target\polyglot\builder\Builder")
00:00:26 verbose #34 > MSBuild version 17.10.0-preview-24101-01+07fd5d51f for .NET
00:00:26 verbose #35 >   Determining projects to restore...
00:00:27 verbose #36 >   Restored C:\home\git\polyglot\target\polyglot\builder\Builder\Builder.fsproj (in 374 ms).
00:00:27 verbose #37 > C:\Users\